In [1]:
# import json

# # Load the JSON file
# with open("chatgpt_dataset/conversations_3.json", "r", encoding="utf-8") as file:
#     data = json.load(file)

# for x, y in data[3]["mapping"].items():
#     if y["message"] is not None:
#         print(y["message"]["content"]["parts"])

# for x, y in data[3]["mapping"].items():
#     if y["message"] is not None and y["message"]["author"]["role"] != "system":
#         print(data[3]["title"])
#         # make first letter of role uppercase
#         print(y["message"]["author"]["role"].capitalize())
#         print(y["message"]["content"]["parts"])

In [2]:
# import json
# import os
# import multiprocessing as mp
# from tqdm import tqdm  # Optional: to display progress bar

# # Function to process a single file
# def process_file(file):
#     local_conversation = []  # Local list to store conversation for this file
#     with open(file, "r", encoding="utf-8") as f:
#         data = json.load(f)
        
#         for item in data:
#             chat = item["title"]
#             for x, y in item["mapping"].items():
#                 if y["message"] is not None and y["message"]["author"]["role"] != "system":
#                     try:
#                         chat += f" {y['message']['author']['role'].capitalize()}: {y['message']['content']['parts'][0]}"
#                     except Exception as e:
#                         pass
#             local_conversation.append(chat)
#     return local_conversation

# # Function to combine results from all files
# def process_all_files(files):
#     # Use a Pool of workers to process files in parallel
#     with mp.Pool(processes=os.cpu_count()) as pool:
#         # Using `tqdm` to track progress
#         results = list(tqdm(pool.imap(process_file, files), total=len(files)))
    
#     # Combine results into a single conversation list
#     all_conversations = []
#     for result in results:
#         all_conversations.extend(result)
    
#     return all_conversations

# # List of files to process
# files = ["chatgpt_dataset/conversations_1.json", "chatgpt_dataset/conversations_2.json", "chatgpt_dataset/conversations_3.json"]

# # Process the files and get the conversation data
# conversation = process_all_files(files)

In [3]:
import json
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  

conversation = []

# Function to process a single file
def process_file(file):
  conversation = []
  with open(file, "r", encoding="utf-8") as f:
    data = json.load(f)

    for item in data:
      chat = item["title"]
      for x, y in item["mapping"].items():
        if y["message"] is not None and y["message"]["author"]["role"] != "system":
          try:
            if len(y['message']['content']['parts'][0]) > 0:
              chat += f" {y['message']['author']['role'].capitalize()}: {y['message']['content']['parts'][0]}"
          except Exception as e:
            pass
      conversation.append(chat)
    return conversation

# Function to combine results from all files
def process_all_files(files):
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        results = list(tqdm(executor.map(process_file, files), total=len(files)))

    # Combine results into a single conversation list
    all_conversations = []
    for result in results:
        all_conversations.extend(result)

    return all_conversations

# Get the list of all files
files = [os.path.join("chatgpt_dataset", file) for file in os.listdir("chatgpt_dataset")]

# Process the files and get the conversation data
conversation = process_all_files(files)


100%|██████████| 3/3 [00:00<?, ?it/s]


In [4]:
len(conversation)

1078

In [6]:
import numpy as np

file = np.array(conversation)
np.save("conversation.npy", file)

In [7]:
file_loaded = np.load("conversation.npy", allow_pickle=True)
print(file_loaded)

['Saving Pytorch Models User: what is the extension for saving models in pytorch? Assistant: In PyTorch, model files are usually saved with the following extensions:\n\n1. **`.pt` or `.pth`**: These are the most common extensions. PyTorch doesn’t enforce a specific extension, so both `.pt` and `.pth` are used interchangeably. \n\n   - Example usage:\n     ```python\n     torch.save(model.state_dict(), "model.pth")  # saves only model parameters\n     torch.save(model, "model_full.pth")          # saves the entire model\n     ```\n\n2. **`.bin`**: Less common, but sometimes used when saving models to be loaded by other frameworks or APIs that also recognize `.bin` format.\n\n   - Example usage:\n     ```python\n     torch.save(model.state_dict(), "model.bin")\n     ```\n\n**Recommendations**:\n- Use `.pth` or `.pt` when saving and loading PyTorch models.\n- Save **only the state dictionary** (`model.state_dict()`) for efficiency, as it doesn’t require the original code structure to relo

In [8]:
len(file_loaded)

1078